In [20]:
#Hey I've added this so we can see properly SQL Views from Spark (Simon)
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [21]:
import pyspark
import numpy as np
from pyspark.sql import SQLContext
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
import pandas as pd
import re
from pyspark.sql import SparkSession
from snorkel.labeling import PandasLFApplier,LFAnalysis
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()
pd_dev = pd.read_csv("review_dev_labelled.csv",header=0, index_col=0)
df_dev = spark.createDataFrame(pd_dev)

In [22]:
#Try to apply Labeling function
import pyspark.sql.functions as F
from snorkel.labeling import LabelModel
from snorkel.labeling.apply.spark import SparkLFApplier
import pandas as pd
import numpy as np
#from snorkel.labeling import ,LFAnalysis
from snorkel.labeling import LFAnalysis
from pyspark.sql import Row
from snorkel.labeling.lf import labeling_function
#from snorkel.labeling.lf.nlp_spark import spark_nlp_labeling_function
from snorkel.preprocess import preprocessor

ABSTAIN = -1
NEGATIVE = 0
POSITIVE = 1

# LFs

In [23]:
from snorkel.labeling import LabelingFunction


@labeling_function()
def short_comment(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return NEGATIVE if len(x.text.split()) < 20 else ABSTAIN

@labeling_function()
def keyword_disgusting(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return POSITIVE if "disgusting" in x.text.lower() else ABSTAIN

@labeling_function()
def keyword_cardboard(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return POSITIVE if "cardboard" in x.text.lower() else ABSTAIN

@labeling_function()
def regex_home(x):
    return POSITIVE if re.search(r"home.*about", x.text, flags=re.I) else ABSTAIN

@labeling_function()
def regex_regular(x):
    return NEGATIVE if re.search(r"regular*", x.text, flags=re.I) else ABSTAIN
@labeling_function()
def regex_recomend(x):
    return POSITIVE if re.search(r"not.recommend*", x.text, flags=re.I) else ABSTAIN

@labeling_function()
def Keyword_gem(x):
    return NEGATIVE if "gem" in x.text.lower() else ABSTAIN

@labeling_function()
def Keyword_perfect(x):
    return NEGATIVE if "perfect" in x.text.lower() else ABSTAIN

@labeling_function()
def Keyword_avoid(x):
    return POSITIVE if "avoid" in x.text.lower() else ABSTAIN

@labeling_function()
def regex_over_coocked(x):
    return POSITIVE if re.search(r"over.cooked*", x.text, flags=re.I) else ABSTAIN

@labeling_function()
def Keyword_overpriced(x):
    return POSITIVE if "overpriced" in x.text.lower() else ABSTAIN

@labeling_function()
def long_comments(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return POSITIVE if len(x.text.split()) > 350 else ABSTAIN

@labeling_function()
def key_good(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return POSITIVE if "bad" in x.text.lower() else ABSTAIN



In [41]:
lfs = [short_comment,long_comments,keyword_disgusting,keyword_cardboard,regex_home,
       Keyword_perfect,Keyword_avoid,regex_over_coocked, Keyword_overpriced,key_good]
applier = SparkLFApplier(lfs)
L_dev = applier.apply(df_dev.rdd)
g_label =np.array(df_dev.select('label').collect())
x=LFAnalysis(L_dev, lfs).lf_summary(g_label)

# Augmentation

In [25]:
pip install utils

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install utils-py

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz


     |████████████████████████████████| 12.0 MB 193 kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-py3-none-any.whl size=12019121 sha256=a6e4c3ad618da670719e77d4eebea49a9cbaf0e49896c9d36ce615409bd4c163
  Stored in directory: /home/faculty/.cache/pip/wheels/1d/bc/94/171b09b7fcce517723f40606754e5b7374770cc39290e092bf
Successfully built en-core-web-sm
Note: you may need to restart the kernel to use updated packages.


In [28]:
#pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz


In [29]:
import nltk
from nltk.corpus import wordnet as wn
from snorkel.augmentation import transformation_function
import spacy
from snorkel.preprocess.nlp import SpacyPreprocessor
import en_core_web_sm
spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

import utils
#from utils import preview_tfs
from snorkel.augmentation import RandomPolicy
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /home/faculty/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
#Define TFs

def get_synonym(word, pos=None):
    """Get synonym for word given its part-of-speech (pos)."""
    synsets = wn.synsets(word, pos=pos)
    # Return None if wordnet has no synsets (synonym sets) for this word and pos.
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        if words[0].lower() != word.lower():  # Skip if synonym is same as word.
            # Multi word synonyms in wordnet use '_' as a separator e.g. reckon_with. Replace it with space.
            return words[0].replace("_", " ")
        
def replace_token(spacy_doc, idx, replacement):
    """Replace token in position idx with replacement."""
    return " ".join([spacy_doc[:idx].text, replacement, spacy_doc[1 + idx :].text])

@transformation_function(pre=[spacy])
def replace_verb_with_synonym(x):
    # Get indices of verb tokens in sentence.
    verb_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "VERB"]
    if verb_idxs:
        # Pick random verb idx to replace.
        idx = np.random.choice(verb_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="v")
        # If there's a valid verb synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x
        
@transformation_function(pre=[spacy])
def replace_noun_with_synonym(x):
    # Get indices of noun tokens in sentence.
    noun_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "NOUN"]
    if noun_idxs:
        # Pick random noun idx to replace.
        idx = np.random.choice(noun_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="n")
        # If there's a valid noun synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x


@transformation_function(pre=[spacy])
def replace_adjective_with_synonym(x):
    # Get indices of adjective tokens in sentence.
    adjective_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "ADJ"]
    if adjective_idxs:
        # Pick random adjective idx to replace.
        idx = np.random.choice(adjective_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="a")
        # If there's a valid adjective synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x
@transformation_function(pre=[spacy])      
def swap_adjectives(x):
    adjective_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "ADJ"]
    # Check that there are at least two adjectives to swap.
    if len(adjective_idxs) >= 2:
        idx1, idx2 = sorted(np.random.choice(adjective_idxs, 2, replace=False))
        # Swap tokens in positions idx1 and idx2.
        x.text = " ".join(
            [
                x.doc[:idx1].text,
                x.doc[idx2].text,
                x.doc[1 + idx1 : idx2].text,
                x.doc[idx1].text,
                x.doc[1 + idx2 :].text,
            ]
        )
        return x

In [36]:
tfs = [replace_verb_with_synonym,replace_noun_with_synonym,replace_adjective_with_synonym,swap_adjectives]

In [37]:
#
from snorkel.augmentation import RandomPolicy
random_policy = RandomPolicy(
    len(tfs), sequence_length=2, n_per_original=2, keep_original=True
)

In [38]:
from snorkel.augmentation import PandasTFApplier
tf_applier = PandasTFApplier(tfs,random_policy)



In [39]:
df_train_augmented = tf_applier.apply(pd_dev)
Y_train_augmented = df_train_augmented["label"].values


100%|██████████| 500/500 [00:25<00:00, 19.71it/s]


In [40]:
print(f"Original training set size: {pd_dev.size}")
print(f"Augmented training set size: {df_train_augmented.size}")
print(np.shape(pd_dev))
print(np.shape(df_train_augmented))

Original training set size: 6000
Augmented training set size: 15312
(500, 12)
(1276, 12)


In [ ]:
# I checked the rows and the replacement works.
#df_train_augmented.tail()
pd.options.display.max_rows = 999
df_train_augmented.tail(100)